In [2]:
import pandas as pd

In [3]:
colnames_echo=['dataid', 'long', 'lat', 'maxdepth', 'shallow', 'intermediate', 'deep', 'total'] 
df_echo = pd.read_csv('out_echo.csv', names=colnames_echo)
df_sensors = pd.read_csv('out_sensors.csv')

In [4]:
df_echo

,dataid,long,lat,maxdepth,shallow,intermediate,deep,total
0,2021-09-24T20:31:46,32.97595,-16.30862,720,1198.70770,140.88910,4.3165,1343.9133
1,2021-09-24T20:58:53,32.95997,-16.30616,720,1343.86860,484.39330,10.5269,1838.7887
2,2021-09-24T21:26:16,32.94445,-16.30271,720,1359.89950,412.35320,11.7806,1784.0333
3,2021-09-24T21:53:42,32.92920,-16.29935,720,1372.47500,524.38450,12.8590,1909.7185
4,2021-09-24T22:26:25,32.91458,-16.29561,720,1378.46350,553.87400,17.6636,1950.0011
...,...,...,...,...,...,...,...,...
156,2021-09-27T07:33:29,30.69806,-16.79325,720,1997.91760,50.72750,86.5452,2555.1904
157,2021-09-27T07:40:49,30.68182,-16.78871,720,1605.77980,51.12810,101.4355,2078.3434
158,2021-09-27T07:48:11,30.66563,-16.78412,720,1765.37430,66.26640,113.4478,2115.0885
159,2021-09-27T07:55:15,30.64936,-16.77965,720,1652.90510,95.21220,135.4374,2213.5547


In [5]:
df_sensors

,dataid,FerryBox.C3_CDOMFluorescence,FerryBox.C3_CHLAFluorescence,NMEA.Humidity
0,2021-08-20T11:46:00,4.07,2.00,36.849998
1,2021-08-20T11:47:00,4.18,2.02,35.642857
2,2021-08-20T11:48:00,4.10,2.09,36.180000
3,2021-08-20T11:49:00,3.99,2.05,38.714287
4,2021-08-20T11:51:00,3.82,1.83,40.933334
...,...,...,...,...
28163,2021-10-03T10:27:00,-0.25,-0.51,73.266670
28164,2021-10-03T10:28:00,-0.09,-0.51,72.949997
28165,2021-10-03T10:29:00,-0.17,-0.51,72.183334
28166,2021-10-03T10:30:00,0.06,-0.50,71.949997


In [11]:
df_sensors.loc[df_sensors['dataid'].str.contains('2021-09-24T20')]


,dataid,FerryBox.C3_CDOMFluorescence,FerryBox.C3_CHLAFluorescence,NMEA.Humidity
19087,2021-09-24T20:45:00,-0.22,-0.49,68.583336
19088,2021-09-24T20:46:00,-0.13,-0.49,69.400002
19089,2021-09-24T20:47:00,-0.38,-0.51,68.199997
19090,2021-09-24T20:48:00,-0.30,-0.49,68.383331
19091,2021-09-24T20:49:00,-0.27,-0.51,68.828575
19092,2021-09-24T20:50:00,-0.15,-0.49,68.940002
19093,2021-09-24T20:51:00,-0.07,-0.51,67.883331
19094,2021-09-24T20:52:00,-0.25,-0.52,67.199997
19095,2021-09-24T20:53:00,-0.21,-0.50,67.239998
19096,2021-09-24T20:54:00,-0.23,-0.49,68.250000
